# Buildmodel

Build nonlinear time series models from scalar time series data using minimum description length radila basis function formalism

## Sample data

First we generate some typical test data to play with

In [ ]:
include("ChaosFunctions.jl")

In [ ]:
using Plots

In [ ]:
z=lorenzpoints(5000,0.05)

In [ ]:
plot(z[1,:],z[2,:],z[3,:])

In [ ]:
z=rosslerpoints(5000,0.2)

In [ ]:
plot(z[1,:],z[2,:],z[3,:])

In [ ]:
z=itmap(tinkerbell,-[0.7, 0.6],10000)

In [ ]:
plot(z[2,:],z[1,:],linetype=:dots)

In [ ]:
zn=addnoise(z,0.05)

In [ ]:
plot(zn[2,:],zn[1,:],linetype=:dots,markersize=0.1)

In [ ]:
plot(zn[1,:],linetype=:dots,markersize=0.5)

In [ ]:
z=itmap(logistic,1/π,10^4);

In [ ]:
z=itmap(logistic,1/π,10^4);
zn=addnoise(z,0.05)
plot(zn[1:end-1],zn[2:end],linetype=:dots,markersize=0.1)

## Build the model

Import the modelling code, and away we go.

In [ ]:
include("BasisFunctionTypes.jl")
include("Place.jl")

Any valid inline function can be used as a penlaty function (it must operate on globally defined functions in `Place.topdown` and must return a scalar, the algorithm seeks a minimum of this function. Four useful examples follow (description length ala `Rissanen` requires extra computation in the main code to account for parameter precisions.

In [ ]:
#need to make these variables in scope of the current WS
nx=Int64(length(z))
mss=Float64(Inf)
λ=Array{Float64,1}[]
δ=Array{Float64,1}[]
#valid penalty criteria
Schwarz = :(nx*log(mss)+nk*log(nx))
Akaike = :(nx*log(mss)+2*nk)
Rissanen = :(description(mss,λ,δ,nx)) #Rissanen desciption length
Model30 = :(-nk*(nk<=30))
#nx is # of observation (length of data)
#nk is # of parameters (basis functions in model)
#mss is the mean-sum-square model prediction error
#λ are the model parameters and δ their precisions

Next, the dictionary `options` defined model structure and optional modelling parameters (it'll run a produce something even if `options` is empty, but it might be better to populate this somewhat intelligently. A useful example to model the (admittedly rather simple) logistic map is provided

In [ ]:
options=Dict("stopstep"=>10,
    "testdatum"=> 8000,
    "functions"=> (gaussian,tophat),
    "embedding" => Place.vembed([0,1,2,3]),
 #   "embedding" => ([0, 1], [0, 1, 2, 3]),
    "penalty"=> Rissanen,
    "nneighbours"=> 1
    )

And, then, away we go...

In [ ]:
mymodel, X, zout, mdlv = Place.buildmodel(zn,options)

Modelling done, we can make one-step predictions on time series data

In [ ]:
plot(mdlv)

In [ ]:
yt,yp,ep = Place.predict(mymodel, z);

In [ ]:
plot(yt,label="truth")
plot!(yp,label="predict")
plot!(ep,label="error",xlimit=(0,500))

Or, free run simulations

In [ ]:
yp, yt = Place.freerun(mymodel,zn,500)

In [ ]:
plot(yt)
plot!(yp,ylimit=(0,1),xlimit=(0,500))


In [ ]:
plot(yt[1:end-1],yt[2:end],linetype=:dots,markersize=0.1,label="test data")
plot!(yp[1:end-1],yp[2:end],linetype=:dots,markersize=1,label="model simulation",title="attractor")

In [ ]:
z=lorenzpoints(5000,0.05)
y=z[1,:]
yn=addnoise(y,0.05)
plot(yn,linetype=:dots,markersize=0.5,label="noisy data")
plot!(y,label="clean data")

In [ ]:
options=Dict("stopstep"=>10,
    "testdatum"=> 8000,
    "functions"=> (gaussian,tophat),
    "embedding" => Place.vembed([0,1,2,3,6,12]),
 #   "embedding" => ([0, 1], [0, 1, 2, 3]),
    "penalty"=> Rissanen,
    "nneighbours"=> 1
    )

In [ ]:
mymodel, X, zout, mdlv = Place.buildmodel(yn,options)

In [ ]:
plot(mdlv)

In [ ]:
yt,yp,ep = Place.predict(mymodel, yn);
plot(yt,label="truth")
plot!(yp,label="predict")
plot!(ep,label="error",xlimit=(0,500))

In [ ]:
yp, yt = Place.freerun(mymodel,yn,500)
plot(yt,label="test data")
plot!(yp,label="free-run prediction")

In [ ]:
plot(yt[1:end-6],yt[4:end-3],yt[7:end],linetype=:dots,markersize=1,label="test data")
plot!(yp[1:end-6],yp[4:end-3],yp[7:end],label="model simulation",title="attractor")

## Michael Small

Last updated today ... 21/12